In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.output_parsers import CommaSeparatedListOutputParser, DatetimeOutputParser, OutputFixingParser, PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import load_prompt

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                   max_completion_tokens=300,
                   temperature=0.7)

In [4]:
output_parser = CommaSeparatedListOutputParser()

In [5]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [6]:
output_parser.parse("red, blue, white")

['red', 'blue', 'white']

In [7]:
human_template = "{request} /n {format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template=human_template)

In [8]:
human_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, template='{request} /n {format_instructions}'), additional_kwargs={})

In [9]:
chat_prompt = ChatPromptTemplate.from_messages(messages=[human_prompt])

In [10]:
chat_prompt

ChatPromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, template='{request} /n {format_instructions}'), additional_kwargs={})])

In [11]:
chat_prompt.format_prompt(request='give me 5 characteristics of dogs',
                          format_instructions=output_parser.get_format_instructions()).to_messages()

[HumanMessage(content='give me 5 characteristics of dogs /n Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})]

In [12]:
model_request = chat_prompt.format_prompt(request='give me 5 characteristics of dogs',
                                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [13]:
output_parser.parse((model.invoke(model_request).content))

['loyalty',
 'intelligence',
 'playfulness',
 'adaptability',
 'strong sense of smell']

## Datetime Parser

In [14]:
datetime_parser = DatetimeOutputParser()

In [15]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

system_prompt = SystemMessagePromptTemplate.from_template(template="You always reply to questions only in datetime patterns.")

In [16]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [17]:
chat_prompt

ChatPromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You always reply to questions only in datetime patterns.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], input_types={}, partial_variables={}, template='{request}\n{format_instructions}'), additional_kwargs={})])

In [18]:
chat_prompt.format_prompt(request="When was the 13th Amendment ratified in the US?",
                          format_instructions=datetime_parser.get_format_instructions())

ChatPromptValue(messages=[SystemMessage(content='You always reply to questions only in datetime patterns.', additional_kwargs={}, response_metadata={}), HumanMessage(content="When was the 13th Amendment ratified in the US?\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0206-01-21T06:40:28.382610Z, 1444-12-29T14:44:24.597000Z, 1610-01-18T18:15:11.106506Z\n\nReturn ONLY this string, no other words!", additional_kwargs={}, response_metadata={})])

In [19]:
model_prompt = chat_prompt.format_prompt(request="When was the 13th Amendment ratified in the US?",
                          format_instructions=datetime_parser.get_format_instructions()).to_messages()

In [20]:
model_prompt

[SystemMessage(content='You always reply to questions only in datetime patterns.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="When was the 13th Amendment ratified in the US?\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1345-01-24T14:04:49.802940Z, 1710-02-28T13:07:37.483757Z, 1282-07-02T13:12:13.727562Z\n\nReturn ONLY this string, no other words!", additional_kwargs={}, response_metadata={})]

In [21]:
datetime_parser.parse(model.invoke(model_prompt).content)

datetime.datetime(1865, 12, 6, 0, 0)

## OutputFixingParser

In [22]:
new_parser = OutputFixingParser.from_llm(parser=datetime_parser, llm=model)

In [23]:
new_parser.parse(model.invoke(model_prompt).content)

datetime.datetime(1865, 12, 6, 0, 0)

In [24]:
new_parser.parse("My birthday is January 9, 1963.")

datetime.datetime(2023, 10, 5, 12, 34, 56, 789012)

## Pydantic Output Parser

In [25]:
class Scientist(BaseModel):

    name: str = Field(description='Name of a Scientist')
    discoveries: list = Field(description="Python list of discoveries")

In [26]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [27]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [28]:
human_prompt = HumanMessagePromptTemplate.from_template(template="{request}\n{format_instructions}")

In [29]:
chat_prompt = ChatPromptTemplate.from_messages(messages=[human_prompt])

In [30]:
request = chat_prompt.format_prompt(request="Tell me a famous scientist",
                                    format_instructions=parser.get_format_instructions()).to_messages()

In [31]:
request

[HumanMessage(content='Tell me a famous scientist\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}\n```', additional_kwargs={}, response_metadata={})]

In [32]:
print(model.invoke(request).content)

```json
{
  "name": "Albert Einstein",
  "discoveries": [
    "Theory of Relativity",
    "Photoelectric Effect",
    "Mass-energy equivalence (E=mc²)",
    "Brownian Motion"
  ]
}
```


In [33]:
parser.parse(model.invoke(request).content)

Scientist(name='Albert Einstein', discoveries=['Theory of Relativity', 'Photoelectric Effect', 'Mass-energy equivalence (E=mc^2)', 'Brownian Motion'])

 ## Saving and Loading Prompts

In [34]:
myprompt = PromptTemplate(template="Tell me a fact about {planet}",
                          input_variables=["planet"])

In [35]:
myprompt.save("myprompt.json")

In [36]:
loaded_prompt = load_prompt("myprompt.json")
loaded_prompt

PromptTemplate(input_variables=['planet'], input_types={}, partial_variables={}, template='Tell me a fact about {planet}')